In [4]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices[0])>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator

import datetime

In [2]:
batch_size=32
epochs=10
input_shape = (20,21,3)
classes = 5
train_gen = DataGenerator('gesture_dataset.h5')

In [5]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(32)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [6]:
model.fit_generator(generator=train_gen,epochs=epochs)

C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/10
18/18 [==============================] - 12s 7ms/step - loss: 1.5533 - accuracy: 0.2558
Epoch 2/10
18/18 [==============================] - 0s 7ms/step - loss: 1.1803 - accuracy: 0.7838
Epoch 3/10
18/18 [==============================] - 0s 6ms/step - loss: 0.6023 - accuracy: 0.8778
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.2687 - accuracy: 0.9466
Epoch 5/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1406 - accuracy: 0.9745
Epoch 6/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0938 - accuracy: 0.9751
Epoch 7/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0903 - accuracy: 0.9787
Epoch 8/10
18/18 [================

In [8]:
model.save("saved_models/MODEL-{}".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), save_format = "tf")

INFO:tensorflow:Assets written to: saved_models/MODEL-2021-05-24-18-38-48\assets
INFO:tensorflow:Assets written to: saved_models/MODEL-2021-05-24-18-38-48\assets
